In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%run Custom_Dataset.ipynb
%run Nibronet2_Model.ipynb
%run Extra_Utilities.ipynb

In [3]:
from sklearn.model_selection import train_test_split
import os,gc
import cv2
import random
import numpy as np
from PIL import Image, ImageStat
from torch.utils.data import Dataset, DataLoader,random_split
from torch import randperm,unique
from torch._utils import _accumulate
import torchvision.transforms.functional as F
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from skimage import io, transform
import sklearn.metrics as skm
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch import manual_seed,zeros_like,zeros,ones,unique,autograd,device,cuda,cat,save,load,tensor,utils,rand
from torch import sum as torch_sum
manual_seed(17)
random.seed(17)

In [19]:
                                 ##### Detection Datasets & Dataloaders #####

# Data Augmentation
only_image_transforms = transforms.Compose([transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5)])#,transforms.RandomGrayscale(p=0.3)])
                
##### Detection Dataset ####

# batchsize for detection
batch_size_detection = 32

# The whole dataset for detection
dataset_detection = DetectionDataset("/home/user/bhassan/baraa/dataset/data/blob",only_image_transforms,transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]))


### spliting the dataset into 80% for training and 10% validation and 10% testing ###
len_dataset_detection = len(dataset_detection)
len_train_dataset_detection,len_test_dataset_detection = int(len_dataset_detection * 0.8),int(0.1*len_dataset_detection)
len_valid_dataset_detection = len_dataset_detection - (len_train_dataset_detection + len_test_dataset_detection)

## splitting datasets ###
train_dataset_detection, validation_dataset_detection, test_dataset_detection = utils.data.random_split(dataset_detection,[len_train_dataset_detection, len_valid_dataset_detection, len_test_dataset_detection])

### dataloaders for detection ###
train_loader_detection = utils.data.DataLoader(dataset=train_dataset_detection, 
                                           batch_size=batch_size_detection, 
                                           shuffle=True)#,**kwargs)
val_loader_detection = utils.data.DataLoader(dataset=validation_dataset_detection, 
                                           batch_size=batch_size_detection, 
                                           shuffle=True)#,**kwargs)
test_loader_detection = utils.data.DataLoader(dataset=test_dataset_detection, 
                                           batch_size=batch_size_detection, 
                                           shuffle=True)#,**kwargs)

print("for Detection: ", len_train_dataset_detection, " samples for training ", len_valid_dataset_detection, " samples for validation ", len_test_dataset_detection, " samlpes for testing")

for Detection:  7086  samples for training  887  samples for validation  885  samlpes for testing


In [11]:

                                ##### Segmentation Datasets & Dataloaders #####

#### segmentation dataset #####

# batch size for segmentation
batch_size_segmentation = 32

# The whole dataset for segmentation
dataset_segmentation = SegmentationDataset("/home/user/bhassan/baraa/dataset/data/segmentation",only_image_transforms,transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]))


### spliting the dataset into 80% for training and 10% validation and 10% testing ###

len_dataset_segmentation = len(dataset_segmentation)
len_train_dataset_segmentation,len_test_dataset_segmentation = int(len_dataset_segmentation * 0.8),int(0.1*len_dataset_segmentation)
len_valid_dataset_segmentation = len_dataset_segmentation - (len_train_dataset_segmentation + len_test_dataset_segmentation)


# # splitting datasets
train_dataset_segmentation, validation_dataset_segmentation, test_dataset_segmentation = utils.data.random_split(dataset_segmentation,[len_train_dataset_segmentation,len_valid_dataset_segmentation,len_test_dataset_segmentation])

### dataloaders ###
train_loader_segmentation = utils.data.DataLoader(dataset=train_dataset_segmentation, 
                                           batch_size=batch_size_segmentation, 
                                           shuffle=True)#,**kwargs)
val_loader_segmentation = utils.data.DataLoader(dataset=validation_dataset_segmentation, 
                                           batch_size=batch_size_segmentation, 
                                           shuffle=True)#,**kwargs)
test_loader_segmentation = utils.data.DataLoader(dataset=test_dataset_segmentation, 
                                           batch_size=batch_size_segmentation, 
                                           shuffle=True)#,**kwargs)

print("for Segmentation: ", len_train_dataset_segmentation, " samples for training ", len_valid_dataset_segmentation, " samples for validation ", len_test_dataset_segmentation, " samlpes for testing")

for Segmentation:  953  samples for training  120  samples for validation  119  samlpes for testing


In [3]:
current_device = device("cuda:0" if cuda.is_available() else "cpu")
print(current_device)
model = NimbroNet2()
model.to(current_device)

# Criterion for Segmentation
criterion_s = nn.CrossEntropyLoss()#(weight=tensor([0.15,0.15,0.7]).to(current_device))
criterion_s.to(current_device)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)

scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.01,step_size_up = 20000,cycle_momentum=False)

cuda:0


In [5]:
def train_model(loss_d_vector,loss_s_vector,iterations,epochs,init_epoch):
    for epoch in range(init_epoch, epochs+1):
        print("epoch",epoch)
        for iteration in range(iterations):
            
            # batch detection
            images_det,heatmaps = next(iter(train_loader_detection))
            images_det = images_det.to(current_device)
            heatmaps = heatmaps.to(current_device)
            
            # batch segmentation
            images_seg,target = next(iter(train_loader_segmentation))
            images_seg = images_seg.to(current_device)
            target = target.to(current_device)

            # To save memory from exceeding limit
            gc.collect()
            cuda.empty_cache()
            cuda.reset_max_memory_allocated()

            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()

            with cuda.amp.autocast():
                # Forward pass to get detection output
                _,outputsD = model(images_det)
                
                # Forward pass to get segmentation output
                outputsS,_ = model(images_seg)
                
                ### Calculate Loss ###
                
                # Loss of detection
                loss_d = mse_loss_fn(outputsD.float(), heatmaps)
                
                # Loss of segmentation
                loss_s =  (criterion_s(outputsS, target.long()))
                
                # Total loss
                loss = 0.03 * loss_d + loss_s + tv_loss(outputsD,4e-5,[0,1,2]) + tv_loss(outputsS,1e-7,[0,1])
                

            # Getting gradients w.r.t. parameters
            scaler.scale(loss).backward()

            # Updating parameters
            scaler.step(optimizer)
            scaler.update()
            # chaning the learning rate of the cyclic learning rate
            scheduler.step()
        

        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} Loss segmentation:{:.6f} Loss detection {:.6f}'.format(
            epoch, iteration * len(images_det), len(train_loader_detection.dataset),
            100. * float(iteration) / len(train_loader_detection), loss.item(),loss_s.item(),loss_d.item()))
        loss_d_vector.append(loss_d.item())
        loss_s_vector.append(loss_s.item())

    # showing the output after training for one epoch        
#         outputsS,outputsD = model(images_det)
#         outputsD =outputsD[0]


#         plt.imshow(trans(heatmaps[0].cpu()))
#         plt.show()

#         target = target[0]
#         target[target == 0] = 0
#         target[target == 2] = 255 
#         target[target == 1] = 128

#         plt.imshow(target.cpu(), cmap='gray', vmin=0, vmax=255,interpolation='nearest')
#         plt.show()

#         img = outputsS.data.max(1)[1]        
#         img[img == 0] = 0
#         img[img == 2] = 255 
#         img[img == 1] = 128
#         plt.imshow(img[0].cpu(), cmap='gray', vmin=0, vmax=255,interpolation='nearest')
#         plt.show()

#         print(outputsD.max())
#         print(outputsD.min())

#         outputsD[outputsD<0] = 0
#         outputsD[outputsD>1] = 1
#         plt.imshow(trans(outputsD.cpu()))
#         plt.show()



#         inp = images_det[0].cpu()
#         inp = inp.numpy().transpose((1, 2, 0))
#         #mean = np.array([0.3539, 0.3919, 0.2552])
#         #std = np.array([0.0391, 0.0384, 0.0438])
#         mean = np.array(det_mean)
#         std = np.array(det_std)
#         inp = std * inp + mean
#         inp = np.clip(inp, 0, 1)
#         inp = (inp * 255).astype(np.uint8)

#         plt.imshow(inp)
#         plt.show()
        #plt.imshow(transforms.ToPILImage()(images[0].cpu()))
        #plt.show()
        
        if (epoch % 5 == 0):
           path = "/home/user/bhassan/baraa/total_loss_full_batch_epoch" + str(epoch) + ".th"
           save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss_d':loss_d_vector,
                'loss_s':loss_s_vector,
                'DetectionTrainDataLoader': train_loader_detection,
                'SegmentationTrainDataLoader': train_loader_segmentation,
                'DetectionTestDataLoader': test_loader_detection,
                'DetectionValidationDataLoader': val_loader_detection,
                'SegmentationTestDataLoader': test_loader_segmentation,
                'SegmentationValidationDataLoader': val_loader_segmentation
                }, path)

        print("lr = ",optimizer.state_dict()["param_groups"][0]["lr"])
    return loss_d_vector,loss_s_vector

In [ ]:
freezing(model.resnet_encoder_block1.parameters(),False)
freezing(model.resnet_encoder_block2.parameters(),False)
freezing(model.resnet_encoder_block3.parameters(),False)
freezing(model.resnet_encoder_block4.parameters(),False)
loss_d_vector,loss_s_vector = [],[]
loss_d_vector,loss_s_vector = train_model(loss_d_vector,loss_s_vector,200,50,1)

# There was a connection problem in the server so I had to repeat the training from epoch 41

In [8]:
path = "/home/user/bhassan/baraa/total_loss_full_batch_epoch40.th"
checkpoint = load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch_init = checkpoint['epoch']
test_loader_detection = checkpoint['DetectionTestDataLoader']
val_loader_detection = checkpoint['DetectionValidationDataLoader']
test_loader_segmentation = checkpoint['SegmentationTestDataLoader']
val_loader_segmentation = checkpoint['SegmentationValidationDataLoader']
train_loader_detection = checkpoint['DetectionTrainDataLoader']
train_loader_segmentation = checkpoint['SegmentationTrainDataLoader']
loss_d_vector = checkpoint['loss_d']
loss_s_vector = checkpoint['loss_s']
#print(epoch_init)
loss_d_vector,loss_s_vector = train_model(loss_d_vector,loss_s_vector,200,50,epoch_init+1)

epoch 41
Train Epoch: 41 [6368/7086 (90%)]	Loss: 0.288695 Loss segmentation:0.043272 Loss detection 7.874002
lr =  0.001991799999999999
epoch 42
Train Epoch: 42 [6368/7086 (90%)]	Loss: 0.258599 Loss segmentation:0.048363 Loss detection 6.725509
lr =  0.002081800000000001
epoch 43
Train Epoch: 43 [6368/7086 (90%)]	Loss: 0.765309 Loss segmentation:0.436301 Loss detection 10.603941
lr =  0.002171799999999999
epoch 44
Train Epoch: 44 [6368/7086 (90%)]	Loss: 0.589177 Loss segmentation:0.223264 Loss detection 11.805671
lr =  0.0022618000000000013
epoch 45
Train Epoch: 45 [6368/7086 (90%)]	Loss: 0.483413 Loss segmentation:0.166763 Loss detection 10.201948
lr =  0.0023517999999999994
epoch 46
Train Epoch: 46 [6368/7086 (90%)]	Loss: 0.431735 Loss segmentation:0.129757 Loss detection 9.772356
lr =  0.0024418000000000013
epoch 47
Train Epoch: 47 [6368/7086 (90%)]	Loss: 0.425974 Loss segmentation:0.111419 Loss detection 10.182708
lr =  0.0025317999999999994
epoch 48
Train Epoch: 48 [6368/7086 (90%

In [ ]:
freezing(model.resnet_encoder_block1.parameters(),True)
freezing(model.resnet_encoder_block2.parameters(),True)
freezing(model.resnet_encoder_block3.parameters(),True)
freezing(model.resnet_encoder_block4.parameters(),True)
loss_d_vector,loss_s_vector = train_model(loss_d_vector,loss_s_vector,200,150,51)

epoch 51
Train Epoch: 51 [6368/7086 (90%)]	Loss: 0.345014 Loss segmentation:0.066339 Loss detection 9.010564
lr =  0.0028918
epoch 52
Train Epoch: 52 [6368/7086 (90%)]	Loss: 0.342296 Loss segmentation:0.066385 Loss detection 8.921173
lr =  0.002981800000000002
epoch 53
Train Epoch: 53 [6368/7086 (90%)]	Loss: 0.364717 Loss segmentation:0.092232 Loss detection 8.800212
lr =  0.0030718
epoch 54
Train Epoch: 54 [6368/7086 (90%)]	Loss: 0.346673 Loss segmentation:0.073410 Loss detection 8.767645
lr =  0.003161800000000002
epoch 55
Train Epoch: 55 [6368/7086 (90%)]	Loss: 0.345793 Loss segmentation:0.062654 Loss detection 9.141215
lr =  0.0032518
epoch 56
Train Epoch: 56 [6368/7086 (90%)]	Loss: 0.348243 Loss segmentation:0.077004 Loss detection 8.755787
lr =  0.003341800000000002
epoch 57
Train Epoch: 57 [6368/7086 (90%)]	Loss: 0.418540 Loss segmentation:0.072962 Loss detection 11.200232
lr =  0.0034318000000000005
epoch 58
Train Epoch: 58 [6368/7086 (90%)]	Loss: 0.334825 Loss segmentation:0.0

In [ ]:
path = "/home/user/bhassan/baraa/total_loss_full_batch_epoch85.th"
checkpoint = load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch_init = checkpoint['epoch']
test_loader_detection = checkpoint['DetectionTestDataLoader']
val_loader_detection = checkpoint['DetectionValidationDataLoader']
test_loader_segmentation = checkpoint['SegmentationTestDataLoader']
val_loader_segmentation = checkpoint['SegmentationValidationDataLoader']
train_loader_detection = checkpoint['DetectionTrainDataLoader']
train_loader_segmentation = checkpoint['SegmentationTrainDataLoader']
loss_d_vector = checkpoint['loss_d']
loss_s_vector = checkpoint['loss_s']
#print(epoch_init)
loss_d_vector,loss_s_vector = train_model(loss_d_vector,loss_s_vector,200,200,epoch_init+1)

epoch 86


/home/user/bhassan/venv/lib/python3.8/site-packages/torch/cuda/memory.py:271: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


Train Epoch: 86 [6368/7086 (90%)]	Loss: 0.307919 Loss segmentation:0.062854 Loss detection 7.931314
lr =  0.001089999999999998
epoch 87
Train Epoch: 87 [6368/7086 (90%)]	Loss: 0.289582 Loss segmentation:0.070931 Loss detection 7.047453
lr =  0.0011800000000000003
epoch 88
Train Epoch: 88 [6368/7086 (90%)]	Loss: 0.313120 Loss segmentation:0.062137 Loss detection 8.125912
lr =  0.0012699999999999983
epoch 89
Train Epoch: 89 [6368/7086 (90%)]	Loss: 0.293712 Loss segmentation:0.057587 Loss detection 7.623602
lr =  0.0013600000000000003
epoch 90
Train Epoch: 90 [6368/7086 (90%)]	Loss: 0.336843 Loss segmentation:0.070767 Loss detection 8.606990
lr =  0.0014499999999999986
epoch 91
Train Epoch: 91 [6368/7086 (90%)]	Loss: 0.268974 Loss segmentation:0.057177 Loss detection 6.801995
lr =  0.0015400000000000006
epoch 92
Train Epoch: 92 [6368/7086 (90%)]	Loss: 0.285302 Loss segmentation:0.062081 Loss detection 7.192020
lr =  0.0016299999999999986
epoch 93
Train Epoch: 93 [6368/7086 (90%)]	Loss: 0.